In [49]:
import feedparser;
import pika;
import json;
import time;
import mysql.connector;

In [46]:
rss_url = 'https://rss.nytimes.com/services/xml/rss/nyt/AsiaPacific.xml'
feed = feedparser.parse(rss_url)

In [47]:
output = {
    'id':feed.entries[0].id,
    'title':feed.entries[0].title,
    'url':feed.entries[0].link,
    'author':feed.entries[0].author,
    'date':feed.entries[0].published
}

In [42]:
connection = pika.BlockingConnection(pika.ConnectionParameters('192.168.0.202'))
channel = connection.channel()

In [50]:
json.dumps(output)

'{"id": "https://www.nytimes.com/2025/03/29/world/asia/taliban-appears-to-free-american-woman-detained-in-afghanistan.html", "title": "Taliban Appears to Free American Woman Detained in Afghanistan", "url": "https://www.nytimes.com/2025/03/29/world/asia/taliban-appears-to-free-american-woman-detained-in-afghanistan.html", "author": "Eve Sampson", "date": "Sat, 29 Mar 2025 22:48:00 +0000"}'

In [22]:
channel.basic_publish(exchange='news.rss', routing_key='nytime', body=json.dumps(output))


In [44]:
for entrie in feed.entries:
    output = {
        'id':entrie.id,
        'title':entrie.title,
        'url':entrie.link,
        'author':entrie.author,
        'date':entrie.published
    }
    channel.basic_publish(exchange='news.rss', routing_key='nytime', body=json.dumps(output))


In [52]:
conn = mysql.connector.connect(
    host="192.168.0.202",
    port=4000,
    user="root",
    password="password",
    database="news"
)

cursor = conn.cursor()


In [56]:
sql = "INSERT INTO test (name) VALUES (%s)"
val = ["Alice"]
try:
    cursor.execute(sql, val)
    conn.commit()  # Always commit after changes
except Exception as e:
    conn.rollback()
    print("database error:", e)
finally:
    cursor.close()
    conn.close()

OperationalError: 2055: Lost connection to MySQL server at '192.168.0.202:4000', system error: 10038 在一个非套接字上尝试了一个操作。